In [ ]:
# pip install racplusplus

In [1]:
import time
import numpy as np
import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt
# import missingno as mno
import kagglehub
# import racplusplus
# from decimal import Decimal
# from miceforest import ImputationKernel
# from sklearn.experimental import enable_iterative_imputer
# from sklearn.linear_model import LinearRegression, BayesianRidge
# from sklearn.neighbors import KNeighborsRegressor, kneighbors_graph
# from sklearn.tree import DecisionTreeRegressor
from sklearn.impute import SimpleImputer #, IterativeImputer, KNNImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder
# from sklearn.neural_network import MLPRegressor
# from sklearn.cluster import AgglomerativeClustering, Birch
# from sklearn.datasets import make_blobs
from scipy.cluster.hierarchy import linkage # dendrogram

In [2]:
kagglehub.dataset_download('marcellomarcello/private-blood-count')

'/kaggle/input/private-blood-count'

In [ ]:
data = pd.read_csv('/kaggle/input/private-blood-count/30_sep_edit.csv')

In [4]:
data

,WBC(10^3/uL),RBC(10^6/uL),HGB(g/dL),HCT(%),MCV(fL),MCH(pg),MCHC(g/dL),PLT(10^3/uL),RDW-SD(fL),RDW-CV(%),...,[MO-Z(ch)],[NE-WX],[NE-WY],[NE-WZ],[LY-WX],[LY-WY],[LY-WZ],[MO-WX],[MO-WY],[MO-WZ]
0,3.02,2.42,5.6,17.8,73.6,23.1,31.5,87.0,49.0,18.6,...,95.6,357.0,580.0,657.0,772.0,602.0,706.0,368.0,837.0,743.0
1,17.42,5.15,15.1,44.4,86.2,29.3,34.0,552.0,46.2,14.9,...,104.3,332.0,725.0,759.0,725.0,940.0,741.0,366.0,913.0,786.0
2,6.87,4.44,12.1,36.2,81.5,27.3,33.4,242.0,47.9,16.3,...,99.0,349.0,708.0,727.0,735.0,756.0,697.0,316.0,762.0,767.0
3,7.04,4.50,12.0,36.5,81.1,26.7,32.9,227.0,46.7,16.2,...,100.1,365.0,667.0,712.0,754.0,708.0,653.0,368.0,753.0,709.0
4,2.86,2.31,5.6,17.1,74.0,24.2,32.7,88.0,49.6,18.8,...,100.6,318.0,622.0,795.0,771.0,718.0,738.0,374.0,796.0,686.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79736,5.69,4.45,13.1,42.1,94.6,29.4,31.1,145.0,49.2,14.1,...,60.8,297.0,566.0,679.0,446.0,884.0,569.0,248.0,637.0,641.0
79737,11.81,4.38,12.0,37.0,84.5,27.4,32.4,644.0,42.2,13.7,...,64.6,330.0,617.0,736.0,635.0,1163.0,603.0,238.0,669.0,573.0
79738,3.73,3.24,8.9,26.8,82.7,27.5,33.2,104.0,46.1,15.2,...,72.1,395.0,711.0,636.0,594.0,631.0,450.0,183.0,60.0,291.0
79739,8.05,4.54,11.8,37.3,82.2,26.0,31.6,56.0,41.4,14.3,...,63.5,319.0,740.0,734.0,1111.0,1591.0,720.0,271.0,651.0,552.0


In [5]:
good_data = data.loc[data.isna().sum(axis = 1) == 0, :]

# How am I supposed to cluster?
* Data preprocessing methods:
    * First drop all the NaN rows. Then, drop the ---- columns in the labels. Once we get a clustering pipeline ready, we will then work on trying to modify the data (imputing etc.)
* Vanilla agglomerative hierarchical clustering:
    - too memory intensive
    - linkage vs connectivity matrix
* Look for memory efficient hierarchical clustering methods
* Louvain clustering
* Birch -> helpful?
* RAC -> helpful?
* Divisive clustering? Useful?

In [6]:
ld = pd.read_csv('/kaggle/input/private-blood-count/labelled_datapoints.csv')
s = pd.DataFrame(range(1, len(ld.columns) + 1), index = ld.columns, columns = ['col_nos'])
s
# s['[RBC-BF2/M]']

,col_nos
Nickname,1
Analyzer ID,2
Date,3
Time,4
Rack,5
...,...
Unnamed: 356,357
Unnamed: 357,358
Unnamed: 358,359
Unnamed: 359,360


In [7]:
def number_to_alphabet(number):
    alphabet = []
    while number > 0:
        remainder = number % 26
        if remainder == 0:
            remainder = 26
            number -= 1
        alphabet.append(chr(ord('A') + remainder - 1))
        number //= 26
    return ''.join(reversed(alphabet))


s['col_excel'] = s['col_nos'].apply(number_to_alphabet)
print(s)

              col_nos col_excel
Nickname            1         A
Analyzer ID         2         B
Date                3         C
Time                4         D
Rack                5         E
...               ...       ...
Unnamed: 356      357        MS
Unnamed: 357      358        MT
Unnamed: 358      359        MU
Unnamed: 359      360        MV
Unnamed: 360      361        MW

[361 rows x 2 columns]


In [8]:
for c in data.columns:
    print(c + ':' + ' ' * (20-len(c)) + s.loc[c, 'col_excel'])

WBC(10^3/uL):        CO
RBC(10^6/uL):        CQ
HGB(g/dL):           CS
HCT(%):              CU
MCV(fL):             CW
MCH(pg):             CY
MCHC(g/dL):          DA
PLT(10^3/uL):        DC
RDW-SD(fL):          DE
RDW-CV(%):           DG
PDW(fL):             DI
MPV(fL):             DK
P-LCR(%):            DM
PCT(%):              DO
NRBC#(10^3/uL):      DQ
NRBC%(%):            DS
NEUT#(10^3/uL):      DU
LYMPH#(10^3/uL):     DW
MONO#(10^3/uL):      DY
EO#(10^3/uL):        EA
BASO#(10^3/uL):      EC
NEUT%(%):            EE
LYMPH%(%):           EG
MONO%(%):            EI
EO%(%):              EK
BASO%(%):            EM
IG#(10^3/uL):        EO
IG%(%):              EQ
[PLT-I(10^3/uL)]:    FI
MicroR(%):           FK
MacroR(%):           FM
[TNC(10^3/uL)]:      FO
[WBC-N(10^3/uL)]:    FQ
[TNC-N(10^3/uL)]:    FS
[BA-N#(10^3/uL)]:    FU
[BA-N%(%)]:          FW
[WBC-D(10^3/uL)]:    FY
[TNC-D(10^3/uL)]:    GA
[NEUT#&(10^3/uL)]:   GC
[NEUT%&(%)]:         GE
[LYMP#&(10^3/uL)]:   GG
[LYMP%&(%)]:    

In [9]:
labelled_data = pd.read_csv('/kaggle/input/private-blood-count/labelled_necessary_cols.csv')

In [10]:
labelled_data.Diagnosis.nunique()

18

In [11]:
labelled_data

,WBC(10^3/uL),RBC(10^6/uL),HGB(g/dL),HCT(%),MCV(fL),MCH(pg),MCHC(g/dL),PLT(10^3/uL),RDW-SD(fL),RDW-CV(%),...,[NE-WY],[NE-WZ],[LY-WX],[LY-WY],[LY-WZ],[MO-WX],[MO-WY],[MO-WZ],Diagnosis,Final diagnosis
0,99.41,5.19,13.1,40.5,78.0,25.2,32.3,133.0,39.3,14.6,...,700.0,715.0,478.0,1050.0,537.0,302.0,994.0,680.0,Chronic lymphocytic leukemia,NaN
1,21.12,4.10,12.1,36.6,89.3,29.5,33.1,464.0,61.2,18.6,...,653.0,673.0,391.0,894.0,704.0,367.0,983.0,1155.0,Primary myelofibrosis,NaN
2,52.75,4.05,12.4,38.5,95.1,30.6,32.2,208.0,65.4,19.4,...,2137.0,922.0,352.0,759.0,573.0,408.0,1136.0,719.0,Acute myeloid leukemia,NaN
3,95.03,4.08,9.4,32.6,79.9,23.0,28.8,273.0,44.4,15.8,...,600.0,651.0,505.0,1017.0,563.0,287.0,874.0,573.0,Chronic lymphocytic leukemia,NaN
4,98.52,4.21,12.4,41.1,97.6,29.5,30.2,261.0,55.5,15.9,...,2509.0,918.0,555.0,1296.0,780.0,412.0,1571.0,853.0,Chronic myeloid leukemia chronic phase,NaN
5,138.12,3.79,10.8,35.7,94.2,28.5,30.3,713.0,58.4,17.5,...,2335.0,814.0,1213.0,1721.0,992.0,367.0,709.0,1038.0,Chronic myeloid leukemia chronic phase,NaN
6,17.57,1.68,5.8,20.1,119.6,34.5,28.9,68.0,83.2,19.1,...,804.0,815.0,648.0,2697.0,1676.0,301.0,638.0,736.0,Plasma cell leukemia,NaN
7,327.99,1.41,4.3,16.0,113.5,30.5,26.9,29.0,114.7,29.7,...,2728.0,936.0,440.0,1021.0,620.0,431.0,1215.0,856.0,Acute lymphoblastic leukemia,Early T cell precursor Acute lymphoblastic leu...
8,10.58,2.14,7.1,23.4,109.3,33.2,30.3,117.0,78.5,19.7,...,798.0,889.0,652.0,1001.0,720.0,480.0,878.0,840.0,Acute myeloid leukemia,NaN
9,70.83,3.86,7.7,26.4,68.4,19.9,29.2,25.0,58.0,27.9,...,1207.0,983.0,497.0,975.0,649.0,285.0,780.0,634.0,Acute lymphoblastic leukemia,B-Acute lymphoblastic leukemia


In [12]:
nan_cols = list(labelled_data.columns[labelled_data.isna().sum() != 0])
nan_cols.pop()
nan_cols

['RDW-SD(fL)', 'RDW-CV(%)', 'PDW(fL)', 'MPV(fL)', 'P-LCR(%)', 'PCT(%)']

In [13]:
labelled_data['Diagnosis'].value_counts()

Diagnosis
Chronic myeloid leukemia chronic phase              13
Acute lymphoblastic leukemia                        11
Acute myeloid leukemia                               5
Acute leukemia                                       5
Chronic lymphocytic leukemia                         3
Acute myelomonocytic leukemia                        3
Primary myelofibrosis                                2
Myelodysplastic neoplasm with increased blasts-1     2
Plasma cell leukemia                                 1
Multiple myeloma                                     1
Myeloproliferative neoplasm                          1
Aplastic anemia                                      1
Acute leukemia monocytic                             1
Myelodysplastic neoplasm with increased blasts-2     1
Acute myeloid leukemia morphologically monocytic     1
Non Hodgkin lymphoma                                 1
Acute myeloid leukemia non M3                        1
Atypical chronic lymphocytic leukemia                1


In [14]:
le = LabelEncoder()
labelled_data['Encoded'] = le.fit_transform(labelled_data['Diagnosis'])

In [15]:
labelled_data.Encoded.nunique()

18

In [16]:
labelled_data.loc[labelled_data[nan_cols].isna().sum(axis = 1) != 0, nan_cols]

,RDW-SD(fL),RDW-CV(%),PDW(fL),MPV(fL),P-LCR(%),PCT(%)
9,58.0,27.9,NaN,NaN,NaN,NaN
10,62.3,18.7,NaN,NaN,NaN,NaN
11,NaN,NaN,NaN,NaN,NaN,NaN
20,96.7,28.0,NaN,NaN,NaN,NaN
21,56.7,19.4,NaN,NaN,NaN,NaN
22,49.6,16.7,NaN,NaN,NaN,NaN
23,89.6,23.4,NaN,NaN,NaN,NaN
26,49.7,18.1,NaN,NaN,NaN,NaN
28,60.8,17.9,NaN,NaN,NaN,NaN
29,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# labelled_data.loc[labelled_data.astype(str).apply(lambda x: any(x.str.contains('----')), axis = 1), labelled_data.astype(str).apply(lambda x: any(x.str.contains('----')))]

1. We drop these columns.
2. We impute these columns somehow.

### We drop the ---- columns.

In [18]:
labelled_data.isna().sum().sum()

141

In [19]:
first = good_data.copy()
ld = labelled_data.copy()
first.drop(nan_cols, axis = 1, inplace = True)
ld.drop(nan_cols, axis = 1, inplace = True)

In [20]:
# ld.pop('Final diagnosis')
# ld.pop('Diagnosis')
# ld.pop('Encoded')
# ld.isna().sum().sum()

In [21]:
print(first.isna().sum().sum(), ld.isna().sum().sum())

0 45


In [22]:
labelled_data.shape, ld.shape, first.shape

((54, 67), (54, 61), (79240, 58))

In [23]:
good_data.shape

(79240, 64)

In [24]:
first = pd.concat([first, ld], axis = 0)
first.reset_index(drop = True)
# first.isna().sum().sum()

,WBC(10^3/uL),RBC(10^6/uL),HGB(g/dL),HCT(%),MCV(fL),MCH(pg),MCHC(g/dL),PLT(10^3/uL),NRBC#(10^3/uL),NRBC%(%),...,[NE-WZ],[LY-WX],[LY-WY],[LY-WZ],[MO-WX],[MO-WY],[MO-WZ],Diagnosis,Final diagnosis,Encoded
0,3.02,2.42,5.6,17.8,73.6,23.1,31.5,87.0,0.14,4.6,...,657.0,772.0,602.0,706.0,368.0,837.0,743.0,NaN,NaN,NaN
1,17.42,5.15,15.1,44.4,86.2,29.3,34.0,552.0,1.17,6.7,...,759.0,725.0,940.0,741.0,366.0,913.0,786.0,NaN,NaN,NaN
2,6.87,4.44,12.1,36.2,81.5,27.3,33.4,242.0,0.43,6.3,...,727.0,735.0,756.0,697.0,316.0,762.0,767.0,NaN,NaN,NaN
3,7.04,4.50,12.0,36.5,81.1,26.7,32.9,227.0,0.36,5.1,...,712.0,754.0,708.0,653.0,368.0,753.0,709.0,NaN,NaN,NaN
4,2.86,2.31,5.6,17.1,74.0,24.2,32.7,88.0,0.17,5.9,...,795.0,771.0,718.0,738.0,374.0,796.0,686.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79289,288.66,2.85,6.9,22.7,79.6,24.2,30.4,657.0,0.61,0.2,...,931.0,1182.0,3451.0,1041.0,545.0,1058.0,1129.0,Chronic myeloid leukemia chronic phase,NaN,10.0
79290,96.39,3.61,10.5,33.5,92.8,29.1,31.3,754.0,0.23,0.2,...,885.0,602.0,1376.0,925.0,444.0,1121.0,948.0,Chronic myeloid leukemia chronic phase,NaN,10.0
79291,90.98,2.67,8.4,28.3,106.0,31.5,29.7,327.0,3.24,3.6,...,1032.0,651.0,1531.0,880.0,322.0,1411.0,788.0,Chronic myeloid leukemia chronic phase,NaN,10.0
79292,29.27,5.15,14.5,46.4,90.1,28.2,31.3,203.0,0.01,0.0,...,653.0,501.0,1007.0,576.0,245.0,643.0,643.0,Atypical chronic lymphocytic leukemia,NaN,8.0


In [35]:
len(first.Diagnosis.unique())

19

In [36]:
len(labelled_data.Diagnosis.unique())

18

In [33]:
first.to_csv('labelled_first_all_dropped.csv')